# Learning Using NTK - v2.0

In [1]:
#Autocomplete
%config Completer.use_jedi = False

In [2]:
%matplotlib inline
%time

import numpy as np
import matplotlib.pyplot as plt
import os, time, glob
from tqdm.notebook import tqdm
from sklearn import svm
from scipy import signal

Wall time: 0 ns


## Fully Connected

Here we do not assume the width to go to infinty. Instead we fix $H_0$ using its parameters, and learning a convex optimization problem.

More specifically, let $g(x)= \frac{df(x;\theta)}{d\theta}$, then let's learn a linear separator (using SVM) on $g(x)$.

Here we will assume a three-layer fully-connected network:
$$ f(x) = sign(u^T \sigma (V \sigma (Wx)))$$
and compute $g$ for each layer separately.

In three layers we have:
$$ \frac{df}{dW} = u^T diag(\dot\sigma(V\sigma(Wx))) V diag(\dot\sigma(Wx))\cdot(I_d \otimes x)$$
$$ \frac{df}{dV} = u^T diag(\dot\sigma(V\sigma(Wx)))\cdot(I_d\otimes \sigma(Wx))$$
$$ \frac{df}{du} = \sigma(V\sigma(Wx))$$

### Experiments

In [3]:
relu = lambda A: np.maximum(A, 0)
drelu = lambda A: (A > 0).astype(np.float)
sgn = lambda A: (A > 0).astype(np.float)*2 - 1

In [4]:
def calc_zero_one_error (y_est, y):
    return (y_est != y).mean()

In [5]:
def calc_zero_one_network_error (X, y, u, V, W):
    y_est = sgn(u.T @ relu(V @ relu(W @ X))).T
    return calc_zero_one_error(y_est, y)

In [6]:
d = 25
m = 100
m_test = 100000

# ground truth
np.random.seed(124)
X = np.random.randn(d, m)
X_test = np.random.randn(d, m_test)
W_real = np.random.randn(d, d) / np.sqrt(d)
V_real = np.random.randn(d, d) / np.sqrt(d)
u_real = np.random.randn(d, 1) / np.sqrt(d)
y = sgn(u_real.T @ relu(V_real @ relu(W_real @ X))).T
y_test = sgn(u_real.T @ relu(V_real @ relu(W_real @ X_test))).T

In [7]:
def map_to_df_dW(X, u, V, W):
    m = X.shape[1]
    out = np.zeros((W.size, m))
    for s in range(m):
        x = X[:,s]
        a = drelu(V @ relu(W @ x))
        b = drelu(W @ x)
        c = np.tensordot(a,b,axes=0) * V
        d = np.squeeze(u.T @ c)
        res = np.ndarray.flatten(np.tensordot(d,x,axes=0))
        
        out[:, s] = res
    return out

In [8]:
def map_to_df_dV(X, u, V, W):
    m = X.shape[1]
    out = np.zeros((W.size, m))
    for s in range(m):
        x = X[:,s]
        a = drelu(V @ relu(W @ x))
        b = relu(W @ x)
        c = np.squeeze(u) * np.squeeze(a)
        d = np.tensordot(c,b,axes=0)
        res = np.ndarray.flatten(d)
        
        out[:, s] = res
    return out

In [9]:
def map_to_df_du(X, u, V, W):
    return relu(V @ relu(W @ X))

In [10]:
# Learn W
np.random.seed(0)
W_0 = np.random.randn(d, d) / np.sqrt(d)
V_0 = np.random.randn(d, d) / np.sqrt(d)
u_0 = np.random.randn(d, 1) / np.sqrt(d)
X_g = map_to_df_dW(X, u_0, V_0, W_0)
X_test_g = map_to_df_dW(X_test, u_0, V_0, W_0)

In [11]:
%%time

clfW = svm.SVC(kernel='linear', C = 1.0)
clfW.fit(X_g.T, np.squeeze(y))

train_err = calc_zero_one_error(clfW.predict(X_g.T), np.squeeze(y))
test_err = calc_zero_one_error(clfW.predict(X_test_g.T), np.squeeze(y_test))
train_err, test_err

Wall time: 1.31 s


(0.0, 0.31055)

In [12]:
# Check which error to use
get_err = lambda W: [calc_zero_one_network_error(X, y, u_0, V_0, W), calc_zero_one_network_error(X_test, y_test, u_0, V_0, W)]
W_clf = np.reshape(clfW.coef_,[25,25])
[get_err(W_0), get_err(W_clf), get_err(-W_clf), get_err(W_0 * W_clf), get_err(- W_0 * W_clf)]

[[0.47, 0.57134],
 [0.34, 0.44753],
 [0.69, 0.67522],
 [0.62, 0.6172],
 [0.45, 0.52883]]

It seems that the parameters I should use for the new W is hyperplane learned by the SVM.

In [13]:
%%time

# Learn V
np.random.seed(1)
V_0 = np.random.randn(d, d) / np.sqrt(d)
u_0 = np.random.randn(d, 1) / np.sqrt(d)
X_g = map_to_df_dV(X, u_0, V_0, W_clf)
X_test_g = map_to_df_dV(X_test, u_0, V_0, W_clf)

clfV = svm.SVC(kernel='linear', C = 1.0)
clfV.fit(X_g.T, np.squeeze(y))

train_err = calc_zero_one_error(clfV.predict(X_g.T), np.squeeze(y))
test_err = calc_zero_one_error(clfV.predict(X_test_g.T), np.squeeze(y_test))
train_err, test_err

Wall time: 4.54 s


(0.01, 0.3046)

In [14]:
# Check which error to use
get_err = lambda V: [calc_zero_one_network_error(X, y, u_0, V, W_clf), calc_zero_one_network_error(X_test, y_test, u_0, V, W_clf)]
V_clf = np.reshape(clfV.coef_,[25,25])
[get_err(V_0), get_err(V_clf), get_err(-V_clf), get_err(V_0 * V_clf), get_err(- V_0 * V_clf)]

[[0.39, 0.43147],
 [0.32, 0.41944],
 [0.63, 0.52795],
 [0.28, 0.35861],
 [0.65, 0.5797]]

In [15]:
%%time

# Learn u
np.random.seed(2)
u_0 = np.random.randn(d, 1) / np.sqrt(d)
X_g = map_to_df_du(X, u_0, V_clf, W_clf)
X_test_g = map_to_df_du(X_test, u_0, V_clf, W_clf)

clfu = svm.SVC(kernel='linear', C = 1.0)
clfu.fit(X_g.T, np.squeeze(y))

train_err = calc_zero_one_error(clfu.predict(X_g.T), np.squeeze(y))
test_err = calc_zero_one_error(clfu.predict(X_test_g.T), np.squeeze(y_test))
train_err, test_err

Wall time: 191 ms


(0.1, 0.30021)

In [16]:
# Check which error to use
get_err = lambda u: [calc_zero_one_network_error(X, y, u, V_clf, W_clf), calc_zero_one_network_error(X_test, y_test, u, V_clf, W_clf)]
u_clf = np.reshape(clfu.coef_,[25,1])
[get_err(u_0), get_err(u_clf), get_err(-u_clf), get_err(u_0 * u_clf), get_err(- u_0 * u_clf)]

[[0.51, 0.44247],
 [0.24, 0.36317],
 [0.76, 0.63683],
 [0.28, 0.36707],
 [0.72, 0.63293]]

## Learning with the Loss

Maybe we should not use $g(x)= \frac{df(x;\theta)}{d\theta}$,
but rather
$$g(x)=-\frac{d\ell}{d\theta} = -\frac{d\ell}{df(x;\theta)} \cdot \frac{df(x;\theta)}{d\theta}.$$

Let's try this.

We will assume that $y\in\{0,1\}$, hence use the cross entropy loss:
$$ \ell(f, (x,y))=-y \log f(x) -(1-y)\log(1-f(x))$$
$$ \frac{d\ell}{df} = -\frac{y}{f(x)} + \frac{1-y}{1-f(x)}. $$

Instead of sign, I will use softmax:
$$ softmax(f, x) = \frac{1}{1+\exp( -f(x))} $$
$$ \frac{d softmax}{df} = softmax(f)(1-softmax(f))$$

### Experiments

In [17]:
def calc_crossentropy_error (y_est, y):
    y = y.copy()
    y[y==-1] = 0
    return (-y * np.log(y_est) - (1-y) * np.log(1-y_est)).mean()

In [18]:
def calc_crossentropy_network_error (X, y, u, V, W):
    y_est = calc_softmax_network (X, u, V, W)
    return calc_crossentropy_error(y_est, y)

In [19]:
def calc_gradient_crossentropy_network_error (X, y, u, V, W):
    y_est = calc_softmax_network (X, u, V, W)
    return np.squeeze(-y / y_est + (1-y)/(1-y_est))

In [20]:
def calc_softmax (y):
    return (1 + np.exp(-y))**(-1)

In [21]:
def calc_softmax_network (X, u, V, W):
    y_est = (u.T @ relu(V @ relu(W @ X))).T
    return calc_softmax(y_est)

In [22]:
def calc_gradient_softmax_network (X, u, V, W):
    softmax = calc_softmax_network(X, u, V, W)
    return np.squeeze(softmax * (1 - softmax))

In [30]:
%%time

# Learn W
np.random.seed(0)
W_0 = np.random.randn(d, d) / np.sqrt(d)
V_0 = np.random.randn(d, d) / np.sqrt(d)
u_0 = np.random.randn(d, 1) / np.sqrt(d)
X_g = -map_to_df_dW(X, u_0, V_0, W_0) * \
      calc_gradient_crossentropy_network_error(X, y, u_0, V_0, W_0) * \
      calc_gradient_softmax_network(X, u_0, V_0, W_0)
X_test_g = -map_to_df_dW(X_test, u_0, V_0, W_0) * \
           calc_gradient_crossentropy_network_error(X_test, y_test, u_0, V_0, W_0) * \
           calc_gradient_softmax_network(X_test, u_0, V_0, W_0)

clfW = svm.SVC(kernel='linear', C = 1.0)
clfW.fit(X_g.T, np.squeeze(y))

train_err = calc_zero_one_error(clfW.predict(X_g.T), np.squeeze(y))
test_err = calc_zero_one_error(clfW.predict(X_test_g.T), np.squeeze(y_test))
train_err, test_err

Wall time: 7.48 s


(0.0, 0.22956)

In [31]:
# Check which error to use
get_err = lambda W: [calc_crossentropy_network_error(X, y, u_0, V_0, W), calc_crossentropy_network_error(X_test, y_test, u_0, V_0, W)]
W_clf = np.reshape(clfW.coef_,[25,25])
[get_err(W_0), get_err(W_clf), get_err(-W_clf), get_err(W_0 * W_clf), get_err(- W_0 * W_clf)]

[[0.7735587873306053, 0.8269217247399877],
 [0.8265467137683745, 0.7898671859151777],
 [0.7638620509773334, 0.786877639451744],
 [0.6954339810124139, 0.7034974596662961],
 [0.7030669520691849, 0.7186746424740204]]

In [32]:
%%time

# Learn V
np.random.seed(1)
V_0 = np.random.randn(d, d) / np.sqrt(d)
u_0 = np.random.randn(d, 1) / np.sqrt(d)
X_g = -map_to_df_dV(X, u_0, V_0, W_clf) * \
      calc_gradient_crossentropy_network_error(X, y, u_0, V_0, W_clf) * \
      calc_gradient_softmax_network(X, u_0, V_0, W_0)
X_test_g = -map_to_df_dV(X_test, u_0, V_0, W_clf) * \
      calc_gradient_crossentropy_network_error(X_test, y_test, u_0, V_0, W_clf) * \
      calc_gradient_softmax_network(X_test, u_0, V_0, W_0)

clfV = svm.SVC(kernel='linear', C = 1.0)
clfV.fit(X_g.T, np.squeeze(y))

train_err = calc_zero_one_error(clfV.predict(X_g.T), np.squeeze(y))
test_err = calc_zero_one_error(clfV.predict(X_test_g.T), np.squeeze(y_test))
train_err, test_err

Wall time: 4.96 s


(0.0, 0.01792)

In [33]:
# Check which error to use
get_err = lambda V: [calc_crossentropy_network_error(X, y, u_0, V, W_clf), calc_crossentropy_network_error(X_test, y_test, u_0, V, W_clf)]
V_clf = np.reshape(clfV.coef_,[25,25])
[get_err(V_0), get_err(V_clf), get_err(-V_clf), get_err(V_0 * V_clf), get_err(- V_0 * V_clf)]

[[0.6856788292584675, 0.6931425368356187],
 [0.7818373022174729, 0.7729875979267471],
 [0.7716297092144465, 0.7353440492859961],
 [0.6961251651139589, 0.6965295774631061],
 [0.6861909905689507, 0.6871236333830937]]

In [34]:
%%time

# Learn u
np.random.seed(2)
u_0 = np.random.randn(d, 1) / np.sqrt(d)
X_g = -map_to_df_du(X, u_0, V_clf, W_clf) * \
      calc_gradient_crossentropy_network_error(X, y, u_0, V_clf, W_clf) * \
      calc_gradient_softmax_network(X, u_0, V_0, W_0)
X_test_g = -map_to_df_du(X_test, u_0, V_clf, W_clf) * \
      calc_gradient_crossentropy_network_error(X_test, y_test, u_0, V_clf, W_clf) * \
      calc_gradient_softmax_network(X_test, u_0, V_0, W_0)

clfu = svm.SVC(kernel='linear', C = 1.0)
clfu.fit(X_g.T, np.squeeze(y))

train_err = calc_zero_one_error(clfu.predict(X_g.T), np.squeeze(y))
test_err = calc_zero_one_error(clfu.predict(X_test_g.T), np.squeeze(y_test))
train_err, test_err

Wall time: 247 ms


(0.0, 0.00341)

In [35]:
# Check which error to use
get_err = lambda u: [calc_crossentropy_network_error(X, y, u, V_clf, W_clf), calc_crossentropy_network_error(X_test, y_test, u, V_clf, W_clf)]
u_clf = np.reshape(clfu.coef_,[25,1])
[get_err(u_0), get_err(u_clf), get_err(-u_clf), get_err(u_0 * u_clf), get_err(- u_0 * u_clf)]

[[0.7033599215990398, 0.7031014294740716],
 [1.1398539000227343, 1.0767954791426393],
 [0.8833870837728033, 0.8169911536060205],
 [0.700110543274313, 0.6996878461035808],
 [0.6878400351976741, 0.6879091706266139]]